In [8]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import datetime
import importlib

import numpy as np
import wgan_regression

importlib.reload(wgan_regression)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

%matplotlib qt

In [9]:
VERSION = "v1"
MODEL_TYPE = "LOWER_BAND"

pdata_folder = os.path.abspath("./../processed_data/chorus_neural_network/")
rbsp_chorus_folder = os.path.join(pdata_folder, "observed_chorus")
output_folder = os.path.join(pdata_folder, "models", VERSION)


dataset = np.load(
    file=os.path.join(output_folder, rf"total_dataset_{VERSION}_{MODEL_TYPE}.npz")
)

X_conditional = dataset["X_conditional"]
X_convolutional = dataset["X_convolutional"]
y = dataset["y"]

t_data = X_conditional[:, -1]
X_cond = X_conditional[:, :-1]

dataset.close()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Dev\\Research\\Research_Tools\\processed_data\\chorus_neural_network\\models\\v1\\total_dataset_v1_LOWER_BAND.npz'

In [10]:
print(np.sum(y == 0))
print(y.shape)

print(y.shape[0] - np.sum(y == 0))

9304643
(9852471,)
547828


In [11]:
print(X_conditional.shape)
print(X_cond.shape)
print(X_convolutional.shape)
print(t_data.shape)
print(y.shape)

(9852471, 7)
(9852471, 6)
(9852471, 512)
(9852471,)
(9852471,)


In [12]:
import datetime

def days_since_timestamp(timestamp):
  """Calculates the number of days since a Unix timestamp.

    Args:
        timestamp: The Unix timestamp (in seconds).

    Returns:
        The number of days since the timestamp as an integer.
  """
  dt_object = datetime.datetime.fromtimestamp(timestamp)
  today = datetime.datetime.now()
  time_difference = today - dt_object
  return time_difference.days

In [8]:
t_ref = datetime.datetime(year = 2012, month = 1, day = 1)
num_days = []
for t in t_data:

    dt = datetime.datetime.fromtimestamp(t)
    del_t = dt - t_ref
    num_days.append(int(del_t.days))

num_days = np.asarray(num_days)

In [28]:
noise_dim = 4  # Noise dimension
condition_dim = 6  # Number of conditional coordinates
output_dim = 1  # Regression output dimension
hidden_dim = 128
num_vectors = 1  # Number of time series vectors
time_steps = 512  # Length of each time series
batch_size = 32
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize models
generator = wgan_regression.Generator(noise_dim, condition_dim, output_dim, hidden_dim, num_vectors).to(device)
critic = wgan_regression.Critic(output_dim, condition_dim, hidden_dim, num_vectors).to(device)

# Dummy dataset (replace with your data)
time_series_data = X_convolutional
coordinates =  X_cond
targets = y
dataset = wgan_regression.TimeSeriesDataset(time_series_data, coordinates, targets)
dataloader = wgan_regression.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the model
wgan_regression.train_wgan_gp(generator, critic, dataloader, noise_dim, condition_dim, num_epochs, device)

NameError: name 'Generator' is not defined